In [12]:
import numpy as np
import struct
import matplotlib.pyplot as plt

Границы зон затопления определяются двумя кусочно-линейными функциями. Область ниже кусочно-линейной функции определяемой наборами точек x_zon_1 и y_zon_1 затапливается из р. Ахтуба. Область выше кусочно-линейной функции определяемой наборами точек x_zon_2 и y_zon_2 затапливается из р. Волга. Область между двумя кусочно-линейными функциями затапливается из р. Ахтуба и р. Волга.

In [13]:
x_zon_1=[780, 540, 500, 402, 380, 300, 285, 284]
y_zon_1=[371, 370, 480, 505, 600, 636, 675, 940]
x_zon_2=[760, 500, 460, 362, 360, 240, 230, 229]
y_zon_2=[311, 320, 430, 455, 590, 626, 675, 940]

In [14]:
#Правый берег р. Волга
x_volga1=np.array([362,343,303,302,229,190,154,121,115,91,70,40,27,25])
x_volga2=np.array([26,33,46,75,81,109, 135,191,196,222,247,263,277,298,329,333,359,376,381,426,485,517,565,601,636,730,847,883,944])
y_volga1=np.array([940, 914,863,860,761,691,641,609,602,576,543,499,466,433])
y_volga2=np.array([424,390,361,323,320,295, 275,258,254,264,288,295,296,289,260,248,211,191,181,158,157,163,149,133,101,35,40,49,88])
y_akhtuba=np.array([783,787,806,779,772,774,733,703,688,682,676,674,697,701,700,670,613,598,589,590,599,623,618,615,604,575,563,575,582,603,601,607,627,633,623,622,658])
x_akhtuba=np.array([341,376,400,478,509,530,529,514,515,520,526,544,571,575,592,622,637,647,661,670,679,706,727,739,751,755,776,806,811,806,821,839,844,856,883,895,944])


In [15]:
def ReadFile(fname):
    """

    """
    try:
        f=open(fname, "rb")
        C=f.read(4).decode("utf-8")
        #print(C)
        N_x=int.from_bytes(f.read(2), "little") # кол-во ячеек по Х
        #print(N_x)
        N_y=int.from_bytes(f.read(2), "little") # кол-во ячеек по Y
        #print(N_y)
        P=struct.unpack('6d', f.read(6*8))
        #print(P)
        Matr = np.zeros ((N_x, N_y))
        # далее читаем матрицу построчно и записываем в Matr
        for i in range(N_x):
            for j in range(N_y):
                temp=struct.unpack('f', f.read(4))
                Matr[i][j]=temp[0]                      
    except IOError:
        print("An IOError has occurred!")
    finally:
        f.close()
    return Matr,P, N_x, N_y

def WriteFile(fname, id_surfer,N_x, N_y, AxisXY,  Matrix):
    """
    id_surfer (тип char) идентификатор (4-байтовая идентификационная строка 'DSBB', которая идентифицирует файл как файл сетки Surfer6binary)
    N_x (тип int) количество линий сетки по оси X (столбцов)
    N_y (тип int) количество линий сетки по оси Y (строк)
    AxisXY=[x_min, x_max, y_min, y_max, z_min, x_max], где  
    x_min (тип double) минимальное значение X сетки,
    x_max (тип double) максимальное значение X сетки,
    y_min (тип double) минимальное значение Y сетки,
    y_max (тип double) максимальное значение Y сетки,
    z_min (тип double) минимальное значение Z сетки,
    z_max (тип double) максимальное значение Z сетки,
    Matr
    """
    try:
        f=open(fname, "wb")
        f.write(id_surfer.encode("utf-8"))
        f.write(struct.pack('h', N_x))
        f.write(struct.pack('h', N_y))
        f.write(struct.pack('6d', AxisXY[0], AxisXY[1], AxisXY[2], AxisXY[3],  np.min(Matrix), np.max(Matrix)))
        for i in range(N_x):
            for j in range(N_y):
                f.write(struct.pack('f', Matrix[i][j]))        
    except IOError:
        print("An IOError has occurred!")
    finally:
        f.close()

In [16]:
def LineEquation (x_a, y_a, x_b, y_b, x):
    """
    Уравнение прямой проходящей через точки (x_a, y_a) и (x_b, y_b)
    """
    return ((y_b-y_a)/(x_b-x_a))*(x-x_a)+y_a
def PiecewiseLinearFunction (x_array, y_array, x):
    """
    С помощью уравнения прямой проходящей через 
    две точки А(x_array[i],y_array[i]) и B(x_array[i+1],y_array[i+1])
    задаем кусочно-линейную функцию для заданных наборов точек
    x_array, y_array
    """
    for i in range(len(x_array)-1):
        if x_array[i+1] <= x <= x_array[i]:
            return ((y_array[i+1]-y_array[i])/(x_array[i+1]-x_array[i]))*(x-x_array[i])+y_array[i]

In [17]:
def FloodingFromVolga(x_zon_2,  y_zon_2, x_0, y_0):
    """
    Проверка затапливается ли точка (x_0,y_0) из реки Волга.
    """
    
    if y_0 <= np.max(y_zon_2):
        return np.interp(x_0, x_zon_2[::-1], y_zon_2[::-1]) > y_0
def FloodingFromAkhtuba(x_zon_1,  y_zon_1, x_0, y_0):
    """
    Проверка затапливается ли точка (x_0,y_0) из реки Ахтуба.
    """
    if y_0 <= np.max(y_zon_1):
        return np.interp(x_0, x_zon_1[::-1], y_zon_1[::-1]) < y_0

In [7]:

x_volga=np.hstack([x_volga1, x_volga2])
y_volga=np.hstack([y_volga1, y_volga2])

def LineLength (x_array, y_array):
    """
    Вычисление длины ломанной-линии.
    """
    
    length=0
    for i in range(len(x_array)-1):
        length=length+50*np.linalg.norm(np.array([x_array[i+1],y_array[i+1]])-np.array([x_array[i],y_array[i]]))
        print (f'{i+1}, {x_array[i+1], y_array[i+1]}, {length}')
LineLength (x_akhtuba, y_akhtuba)

def MinLength (length_zone, x_array, y_array):
    minLen=4000
    length_channel=0
    for i in range(len(x_array)-1):
        length_channel=length_channel+50*np.linalg.norm(np.array([x_array[i+1],y_array[i+1]])-np.array([x_array[i],y_array[i]]))
        if np.abs(length_zone-length_channel)<minLen:
            minLen=np.abs(length_zone-length_channel)
            point_number=i+1
    return point_number
#print(f'{MinLength (57000, x_array, y_array)}, {x_array[MinLength (57000, x_array, y_array)]}, {y_array[MinLength (57000, x_array, y_array)]}')

def PointNumber (x_zone, y_zone, x, y):
    for i in range(len(x_zone)-1):
        if (x_zone[i+1] <= x <= x_zone[i] and np.abs(y_zone[i] - y_zone[i+1])<15) or y_zone[i] <= y <= y_zone[i+1]:
            return i
            
        
def PointVolgaChannel(x_channel, y_channel, x_zone, y_zone, length_zone):
    """
     Вычисляются длины участков для склейки.
     Параметры??!!
    """
    x_len=[]
    y_len=[]
    for i in range(len(length_zone)-1):
        x_len=np.append(x_len, x_channel[MinLength (length_zone[i], x_channel, y_channel)])
        y_len=np.append(y_len, y_channel[MinLength (length_zone[i], x_channel, y_channel)])
    return x_len, y_len

def generator (Q, delta_Q, number_zones, day, N_x, N_y):
    """
    Для теста склейки
    """
    
    M = np.zeros ((N_x, N_y))
    for k in range(Q,Q+number_zones*delta_Q,delta_Q):
        for i in range(N_x):
            for j in range(N_y):
                M[i][j]=k
        fname="Q_"+str(int(k/10))+"_day_"+str(day)+".grd"
        WriteFile(fname, "DSBB", 944, 944, PP,   M)





1, (376, 787), 1761.3914953808535
2, (400, 806), 3291.91428188225
3, (478, 779), 7418.959229541531
4, (509, 772), 9007.984087748602
5, (530, 774), 10062.73524323505
6, (529, 733), 12113.344908676037
7, (514, 703), 13790.39589180088
8, (515, 688), 14542.060710719525
9, (520, 682), 14932.573194514858
10, (526, 676), 15356.837263226787
11, (544, 674), 16262.375777040528
12, (571, 697), 18035.790562137347
13, (575, 701), 18318.633274611966
14, (592, 700), 19170.102592908286
15, (622, 670), 21291.422936467927
16, (637, 613), 24238.455342126203
17, (647, 598), 25139.8431609922
18, (661, 589), 25972.009009846865
19, (670, 590), 26424.778266753736
20, (679, 599), 27061.174369821627
21, (706, 623), 28867.413556640473
22, (727, 618), 29946.765213886618
23, (739, 615), 30565.231057729266
24, (751, 604), 31379.17208753425
25, (755, 575), 32842.9002043647
26, (776, 563), 34052.23886660948
27, (806, 575), 35667.78830874983
28, (811, 582), 36097.90457210196
29, (806, 603), 37177.2562293481
30, (821, 

In [10]:
length_zone_Volga=np.array([5000,21000,37000,55000,88000])
length_zone_Akhtuba=np.array([9000,18000,27000,36000,45000])


x_length_volga, y_length_volga = PointVolgaChannel(x_volga, y_volga, x_zon_2, y_zon_2, length_zone_Volga)
x_length_akhtuba, y_length_akhtuba = PointVolgaChannel(x_akhtuba, y_akhtuba, x_zon_2, y_zon_2, length_zone_Akhtuba)

x_zonVA=[230, 230, 360, 460]
y_zonVA=[860, 675, 590, 430]

x_zonA=[230, 230, 360, 460]
y_zonA=[860, 675, 590, 430]

print(x_length_volga)
print(y_length_volga)

print(x_length_akhtuba)
print(y_length_akhtuba)




def LoadingMaps (Q_array, number_zones, day, path):
    N_x=944
    N_y=944
    H = np.zeros ((number_zones, N_x, N_y))
    for zone in range(number_zones):
        Q_str=str(int((Q_array[zone])/10))   
        file_name=path+"расчет_2023_манинг_q"+Q_str+"_30day/H_   "+str(day)+".grd"
        H[zone][::][::],PP, N_x, N_y=ReadFile(file_name)
    return H


def GluingCardsOLD (x_zon_1,  y_zon_1, x_zon_2,  y_zon_2, x_length_volga,x_length_akhtuba, N_x, N_y, H_volga, H_volgaakhtuba, H_akhtuba):
    H_gluing = np.zeros ((N_x, N_y))
    for y in range(N_y):
        for x in range (N_x):
            if FloodingFromAkhtuba(x_zon_1,  y_zon_1, x, y):
                if LineEquation (x_length_akhtuba[0], y_length_akhtuba[0], x_zonA[0], y_zonA[0], x)<y:
                    H_gluing[y][x]=H_akhtuba[0][y][x]
                if LineEquation (x_length_akhtuba[len(x_length_akhtuba)-1], y_length_akhtuba[len(x_length_akhtuba)-1], x_zonA[len(x_length_akhtuba)-1], y_zonA[len(x_length_akhtuba)-1], x)>y:
                    H_gluing[y][x]=H_akhtuba[len(x_length_akhtuba)][y][x]
                for point in range(1,len(x_length_akhtuba)):
                    if LineEquation (x_length_akhtuba[point], y_length_akhtuba[point], x_zonA[point], y_zonA[point], x) <= y <= LineEquation (x_length_akhtuba[point-1], y_length_akhtuba[point-1], x_zonA[point-1], y_zonA[point-1], x):
                        H_gluing[y][x]=H_akhtuba[point][y][x] 
            elif FloodingFromVolga(x_zon_2,  y_zon_2, x, y):
                if LineEquation (x_length_volga[0], y_length_volga[0], x_zonVA[0], y_zonVA[0], x)<y:
                    H_gluing[y][x]=H_volga[0][y][x]
                if LineEquation (x_length_volga[len(x_length_volga)-1], y_length_volga[len(x_length_volga)-1], x_zonVA[len(x_length_volga)-1], y_zonVA[len(x_length_volga)-1], x)>y:
                    H_gluing[y][x]=H_volga[len(x_length_volga)][y][x]
                for point in range(1,len(x_length_volga)):
                    if LineEquation (x_length_volga[point], y_length_volga[point], x_zonVA[point], y_zonVA[point], x) <= y <= LineEquation (x_length_volga[point-1], y_length_volga[point-1], x_zonVA[point-1], y_zonVA[point-1], x):
                        H_gluing[y][x]=H_volga[point][y][x]              
            else:
                if LineEquation (x_length_akhtuba[0], y_length_akhtuba[0], x_zonA[0], y_zonA[0], x)<y:
                    H_gluing[y][x]=H_akhtuba[0][y][x]
                if LineEquation (x_length_akhtuba[len(x_length_akhtuba)-1], y_length_akhtuba[len(x_length_akhtuba)-1], x_zonA[len(x_length_akhtuba)-1], y_zonA[len(x_length_akhtuba)-1], x)>y:
                    H_gluing[y][x]=H_akhtuba[len(x_length_akhtuba)][y][x]
                for point in range(1,len(x_length_akhtuba)):
                    if LineEquation (x_length_akhtuba[point], y_length_akhtuba[point], x_zonA[point], y_zonA[point], x) <= y <= LineEquation (x_length_akhtuba[point-1], y_length_akhtuba[point-1], x_zonA[point-1], y_zonA[point-1], x):
                        H_gluing[y][x]=H_akhtuba[point][y][x]         
    return H_gluing

def SelectionZone(x_length,y_length,x_zon,y_zon,H, x,y):
    if LineEquation (x_length[0], y_length[0], x_zon[0], y_zon[0], x)<y:
        return H[0][y][x]
    if LineEquation (x_length[len(x_length)-1], y_length[len(x_length)-1], x_zon[len(x_length)-1], y_zon[len(x_length)-1], x)>y:
        return H[len(x_length)][y][x]
    for point in range(1,len(x_length)):
        if LineEquation (x_length[point], y_length[point], x_zon[point], y_zon[point], x) <= y <= LineEquation (x_length[point-1], y_length[point-1], x_zon[point-1], y_zon[point-1], x):
            return H[point][y][x]


def GluingCards (x_zon_1,  y_zon_1, x_zon_2,  y_zon_2, x_length_volga,x_length_akhtuba, N_x, N_y, H_volga, H_volgaakhtuba, H_akhtuba):
    H_gluing = np.zeros ((N_x, N_y))
    for y in range(N_y):
        for x in range (N_x):
            if FloodingFromAkhtuba(x_zon_1,  y_zon_1, x, y):
                H_gluing[y][x]=SelectionZone(x_length_akhtuba,y_length_akhtuba,x_zonA,y_zonA,H_akhtuba, x,y)
            elif FloodingFromVolga(x_zon_2,  y_zon_2, x, y):
                H_gluing[y][x]=SelectionZone(x_length_volga,y_length_volga,x_zonVA,y_zonVA,H_volga, x,y)         
            else:
                H_gluing[y][x]=SelectionZone(x_length_akhtuba,y_length_akhtuba,x_zonA,y_zonA,H_akhtuba, x,y)                     
    return H_gluing


            
    


[302. 115.  75. 359.]
[860. 602. 323. 211.]
[509. 571. 679. 811.]
[772. 697. 599. 582.]


In [18]:
day=14
number_zones_Volga=5
number_zones_Akhtuba=5
path='Z:/Работа/Базовый корпус карт затопления/'
Rel_test,PP, N_x, N_y=ReadFile("relief_base.grd")
file_name_array=['H_3 (1961г).grd', 'H_2 (1991г).grd','H_4 (2053г)BD.grd','H_5(2053г)150м.grd',
                 'H_6(2053г)125м.grd','H_7(2053г)100м.grd','H_8(2053г)75м.grd','H_неуст 2053 БД.grd',
                 'H_9(2053г)50м.grd','H_10(2053)20км.grd','H_11(2053)35км.grd','H_12(2053)7и35км.grd',
                 'H_13.grd','H_14.grd','H_15.grd']

Q_array_volga=np.array([[28250,28000,27750,27500,27250],
                        [26250,26000,25750,25500,25250],
                        [22250,22500,22750,23000,23250],
                        [22000,22250,22500,22750,23000],
                        [21750,22000,22250,22500,22750],
                        [21500,21750,22000,22250,22500],
                        [21250,21500,21750,22000,22250],
                        [24000,24250,24500,24750,25000],
                        [21000,21250,21500,21750,22000],
                        [26000,28250,22000,22250,22000],
                        [24000,26000,28250,22250,22500],
                        [28000,26000,28250,21750,22000],
                        [22250,22500,22750,23000,23250],
                        [22250,22500,22750,23000,23250],
                        [22250,22500,22750,23000,23250]])
Q_array_akhtuba=np.array([np.full(5, 28250),np.full(5, 26250),
                         np.full(5, 22250),np.full(5, 24000),
                         np.full(5, 25000),np.full(5, 26000),
                         np.full(5, 27500),np.full(5, 26000),
                         np.full(5, 28250),np.full(5, 26000),
                         np.full(5, 24000),np.full(5, 28250),
                         [22500,23000,24500,26000,28250],
                         [23000,24000,26000,28250,22500],
                         [23000,24500,27000,26000,28250]])

In [19]:

for i in range(len(file_name_array)):
    H_volga=LoadingMaps(Q_array_volga[i], number_zones_Volga, day, path)
    H_akhtuba=LoadingMaps (Q_array_akhtuba[i], number_zones_Akhtuba, day, path)
    H=GluingCards (x_zon_1,  y_zon_1, x_zon_2,  y_zon_2, x_length_volga, x_length_akhtuba, N_x, N_y, H_volga, H_akhtuba, H_akhtuba)
    write_path=path+"Склейка_карт/"+file_name_array[i]                   
    WriteFile(write_path, "DSBB", 944,944, PP, H)
    

In [65]:
for i in range(len(file_name_array)):
    print(f'volga={Q_array_volga[i]}, akhtuba={Q_array_akhtuba[i]}, file_name={file_name_array[i]} ')

volga=[28250 28000 27750 27500 27250], akhtuba=[28250 28250 28250 28250 28250], file_name=H_3 (1961г).grd 
volga=[26250 26000 25750 25500 25250], akhtuba=[26250 26250 26250 26250 26250], file_name=H_2 (1991г).grd 
volga=[22250 22500 22750 23000 23250], akhtuba=[22250 22250 22250 22250 22250], file_name=H_4 (2053г)BD.grd 
volga=[22000 22250 22500 22750 23000], akhtuba=[24000 24000 24000 24000 24000], file_name=H_5(2053г)150м.grd 
volga=[21750 22000 22250 22500 22750], akhtuba=[25000 25000 25000 25000 25000], file_name=H_6(2053г)125м.grd 
volga=[21500 21750 22000 22250 22500], akhtuba=[26000 26000 26000 26000 26000], file_name=H_7(2053г)100м.grd 
volga=[21250 21500 21750 22000 22250], akhtuba=[27500 27500 27500 27500 27500], file_name=H_8(2053г)75м.grd 
volga=[24000 24250 24500 24750 25000], akhtuba=[25000 25000 25000 25000 25000], file_name=H_неуст 2053 БД.grd 
volga=[21000 21250 21500 21750 22000], akhtuba=[28250 28250 28250 28250 28250], file_name=H_9(2053г)50м.grd 
volga=[26000 28250

In [114]:
for i in range(len(file_name_array)):
    file_name_TestArray=path+"test-2/"+file_name_array[i]  

    TestArray,PP, N_x, N_y=ReadFile(file_name_TestArray)
    file_name_TestArray=path+"test-1/"+file_name_array[i]  
    Test,PP, N_x, N_y=ReadFile(file_name_TestArray)
    count_er=0
    for y in range(N_y):
        for x in range (N_x):
            if Test[y][x]!=TestArray[y][x]:
                count_er=count_er+1
            H[y][x]=10
    print(file_name_TestArray,count_er) 

Z:/Работа/Базовый корпус карт затопления/test-1/H_3 (1961г).grd 2
Z:/Работа/Базовый корпус карт затопления/test-1/H_2 (1991г).grd 2
Z:/Работа/Базовый корпус карт затопления/test-1/H_4 (2053г)BD.grd 2
Z:/Работа/Базовый корпус карт затопления/test-1/H_5(2053г)150м.grd 2
Z:/Работа/Базовый корпус карт затопления/test-1/H_6(2053г)125м.grd 2
Z:/Работа/Базовый корпус карт затопления/test-1/H_7(2053г)100м.grd 2
Z:/Работа/Базовый корпус карт затопления/test-1/H_8(2053г)75м.grd 2
Z:/Работа/Базовый корпус карт затопления/test-1/H_неуст 2053 БД.grd 2
Z:/Работа/Базовый корпус карт затопления/test-1/H_9(2053г)50м.grd 2
Z:/Работа/Базовый корпус карт затопления/test-1/H_10(2053)20км.grd 2
Z:/Работа/Базовый корпус карт затопления/test-1/H_11(2053)35км.grd 2
Z:/Работа/Базовый корпус карт затопления/test-1/H_12(2053)7и35км.grd 2
Z:/Работа/Базовый корпус карт затопления/test-1/H_13.grd 1201
Z:/Работа/Базовый корпус карт затопления/test-1/H_14.grd 1248
Z:/Работа/Базовый корпус карт затопления/test-1/H_15.g

In [103]:
file_name_TestArray=path+"test/"+"H_13.grd" 
H= np.zeros ((N_x, N_y))
TestArray,PP, N_x, N_y=ReadFile(file_name_TestArray)
file_name_TestArray=path+"test-1/"+"H_13.grd"   
Test,PP, N_x, N_y=ReadFile(file_name_TestArray)
count_er=0
for y in range(N_y):
    for x in range (N_x):
        if Test[y][x]!=TestArray[y][x]:
            count_er=count_er+1
            H[y][x]=10
print(file_name_TestArray,count_er)
write_path=path+"test-1/"+"HHH.grd"
WriteFile(write_path, "DSBB", 944,944, PP, H)

Z:/Работа/Базовый корпус карт затопления/test-1/H_13.grd 1201
